In [25]:
from sklearn.metrics import classification_report
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.utils.layer_utils import print_summary
from tensorflow.keras.models import Model as KerasModel
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Layer, \
    Reshape, Concatenate, LeakyReLU
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from pprint import pprint
#from Utility_functions import create_sequence, FreezeBatchNormalization
import pandas as pd
# from Utility_functions import FreezeBatchNormalization

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [26]:
def generate_generator_multiple(generator, dir1,  batch_size, img_height, img_width):
    genX1 = generator.flow_from_directory(dir1,
                                          target_size = (img_height, img_width),
                                          class_mode = 'binary',
                                          batch_size = batch_size,
                                          shuffle = True,
                                          seed = 32)
                                         

    i = 0
    while (True):
        if i < len(genX1.filepaths):
            X1i = genX1.next()
            yield X1i[0], X1i[1]

        i += 1


In [27]:
batch_size = 64
img_height = 256
img_width = 256
train_datagen = ImageDataGenerator(rescale = 1. / 255)
validation_datagen = ImageDataGenerator(rescale = 1. / 255)
test_datagen = ImageDataGenerator(rescale = 1. / 255)

train_generator = generate_generator_multiple(generator = train_datagen,
                                              dir1 = 'split_dataset/train',
                                             
                                              batch_size = batch_size,
                                              img_height = img_height,
                                              img_width = img_width)
validation_generator = generate_generator_multiple(validation_datagen,
                                                   dir1 = 'split_dataset/val',
                                                   
                                                   batch_size = batch_size,
                                                   img_height = img_height,
                                                   img_width = img_width)
# test_generator = (test_datagen,
#                                                    dir1 = 'split_dataset/test',
                                                   
#                                                    batch_size = batch_size,
#                                                    img_height = img_height,
#                                                    img_width = img_width)
test_generator=train_datagen.flow_from_directory('split_dataset/test',
                                          target_size = (img_height, img_width),
                                          class_mode = 'binary',
                                          batch_size = batch_size,
                                          shuffle = False,
                                          seed = 32)
                                         

Found 4421 images belonging to 2 classes.


In [28]:
def get_pred(predictions):
    pred = []
    for p in predictions:
        if p < 0.50:
            pred.append(0)
        else:
            pred.append(1)
    return pred


IMGWIDTH = 256

In [29]:
class Classifier:
    def __init__(self):
        self.model = 0

    def predict(self, x):
        return self.model.predict(x)

    def fit(self, x, y):
        return self.model.train_on_batch(x, y)

    def get_accuracy(self, x, y):
        return self.model.test_on_batch(x, y)

    def load(self, path):
        self.model.load_weights(path)

In [30]:
class Meso4(Classifier):
    def __init__(self, learning_rate = 0.001):
        self.model = self.init_model()
        optimizer = Adam(lr = learning_rate)
        self.model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])

    def init_model(self):
        x = Input(shape = (IMGWIDTH, IMGWIDTH, 3))

        x1 = Conv2D(8, (3, 3), padding = 'same', activation = 'relu')(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size = (2, 2), padding = 'same')(x1)

        x2 = Conv2D(8, (5, 5), padding = 'same', activation = 'relu')(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size = (2, 2), padding = 'same')(x2)

        x3 = Conv2D(16, (5, 5), padding = 'same', activation = 'relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size = (2, 2), padding = 'same')(x3)

        x4 = Conv2D(16, (5, 5), padding = 'same', activation = 'relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size = (4, 4), padding = 'same')(x4)

        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)       
        y = LeakyReLU(alpha = 0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)

        return KerasModel(inputs = x, outputs = y)

In [31]:
model= Meso4()
model.load("./model/MesoNet3.h5")
model=model.model
is_training=True
top_k_layers=19
#model,df=FreezeBatchNormalization(is_training,top_k_layers,model)
optimizer = Adam(lr = 0.001)
model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])

custom_callbacks = [
    EarlyStopping(
        monitor = 'val_loss',
        mode = 'min',
        patience = 5,
        verbose = 1
    ),
ModelCheckpoint(
    filepath="./model/MesoNet3.h5",
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True),
    
CSVLogger("./model/MesoNet3.csv", append=True, separator=',')
#print_summary(model, line_length=115, positions=None, print_fn=None)
]

total_train = 40000
total_val =  4000
model.fit_generator(train_generator,
                    verbose = 1,
                    steps_per_epoch = int(total_train / batch_size),
                    epochs = 30,
                    validation_data = validation_generator,
                    validation_steps = int(total_val / batch_size),
                    callbacks=custom_callbacks)
model.save_weights("model/MesoNet3_50.h5")

print("model save succesfully")

C:\Users\user\AppData\Local\Temp\ipykernel_11336\4126048486.py:30: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Found 35349 images belonging to 2 classes.
Epoch 1/30
625/625 [==============================] - ETA: 0s - loss: 0.1645 - accuracy: 0.7564Found 4417 images belonging to 2 classes.


625/625 [==============================] - 898s 1s/step - loss: 0.1645 - accuracy: 0.7564 - val_loss: 0.1371 - val_accuracy: 0.7966
Epoch 2/30
396/625 [==================>...........] - ETA: 5:24 - loss: 0.1281 - accuracy: 0.8121

KeyboardInterrupt: 

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 8)       224       
                                                                 
 batch_normalization (Batch  (None, 256, 256, 8)       32        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 8)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 8)       1608      
                                                                 
 batch_normalization_1 (Bat  (None, 128, 128, 8)       32    

In [ ]:
import pandas as pd

In [ ]:
# load the saved model that is considered the best
from tensorflow.keras.models import load_model

# load the saved model that is considered the best
model.load_weights('./model/MesoNet3_50.h5')

# Generate predictions
# test_generator.reset()

preds = model.predict(
    test_generator,
    verbose = 1
)

test_results = pd.DataFrame({
    "Filename": test_generator.filenames,
    "Prediction": preds.flatten()
})
print(test_results)

In [ ]:
# Confusion Matrix

from sklearn.metrics import confusion_matrix

# Load the model weights
model.load_weights("./model/MesoNet3_50.h5")

# Generate predictions on the test data
test_predictions = model.predict(test_generator, verbose=1)
test_labels = test_generator.classes

# Convert predictions to binary values
binary_predictions = (test_predictions > 0.5).astype(int)

# Generate confusion matrix
confusion = confusion_matrix(test_labels, binary_predictions)
print("Confusion Matrix:")
print(confusion)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

# Load the model weights
model.load_weights("./model/MesoNet3_50.h5")

# Generate predictions on the test data
test_predictions = model.predict(test_generator, verbose=1)
test_labels = test_generator.classes

# Convert predictions to binary values
binary_predictions = (test_predictions > 0.5).astype(int)

# Generate confusion matrix
confusion = confusion_matrix(test_labels, binary_predictions)

# Define class labels
class_labels = ['real', 'fake']

# Plot confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot the confusion matrix
plot_confusion_matrix(confusion, classes=class_labels, title='Confusion Matrix')
plt.show()

In [ ]:
CMAP = plt.cm.coolwarm

def plot_tf_hist(hist_df):
    
    # instantiate figure
    fig, axes = plt.subplots(1,2 , figsize = (15,6))
    
    # properties  matplotlib.patch.Patch 
    props = dict(boxstyle='round', facecolor='cyan', alpha=0.5)
    facecolor = 'cyan'
    fontsize=12
    
    # Where was min loss
    best = hist_df[hist_df[hist_df.columns[2]] == hist_df[hist_df.columns[2]].min()]
    
    # pick first axis
    ax = axes[0]
    
    y1 = hist_df.columns[0]
    y2 = hist_df.columns[2]

    # Plot all losses
    hist_df.plot(y = [y1, y2], ax = ax, colormap=CMAP)

    # little beautification
    txtFmt = "Loss: \n  train: {:6.4f}\n   test: {:6.4f}"
    txtstr = txtFmt.format(hist_df.iloc[-1][y1],
                           hist_df.iloc[-1][y2]) #text to plot

    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.95, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)
    
    # Mark arrow at lowest
    ax.annotate(f'Min: {best[y2].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y2].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y2].to_numpy()[0]), # location of text 
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow
    
    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);
    
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Loss")
    ax.set_title('Errors')
    ax.grid()
    ax.legend(loc = 'upper left') # model legend to upper left

    # pick second axis
    ax = axes[1]
    
    y1 = hist_df.columns[1]
    y2 = hist_df.columns[3]

    # Plot all losses
    hist_df.plot(y = [y1, y2], ax = ax, colormap=CMAP)

    # little beautification
    txtFmt = "Accuracy: \n  train: {:6.4f}\n  test:  {:6.4f}"
    txtstr = txtFmt.format(hist_df.iloc[-1][y1],
                           hist_df.iloc[-1][y2]) #text to plot

    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.2, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    ax.annotate(f'Best: {best[y2].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y2].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y2].to_numpy()[0]), # location of text 
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow
    
    
    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);
    
    # Labels
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Accuracy")
    ax.set_title('Accuracies')
    ax.grid();
    ax.legend(loc = 'lower left')
    
    plt.tight_layout()

In [ ]:
loss_df = pd.DataFrame(history.history)
loss_df.head()

In [ ]:
plot_tf_hist(loss_df)